<a href="https://colab.research.google.com/github/zambbo/BERT-blog-sentiment-analysis/blob/master/blogger_scoring_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install kss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
import pandas as pd
import datetime
import kss
import re
import os

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### 아래의 경로들은 환경에 따라 바꿔줘야한다.
# 모델이 저장되어 있는 경로
sent_path = '/content/drive/MyDrive/dataset/sentiment_model.pt'
subj_path = '/content/drive/MyDrive/dataset/subjective_model.pt'
# 블로거의 블로거 글들이 저장되어있는 폴더
root_file_path = "/content/drive/MyDrive/blogger detail"
# 최종 스코어링 한 csv파일을 저장할 폴더
result_file_path = "/content/drive/MyDrive/After Scoring"



#데이터 셋을 BERT모델에 알맞은 형태로 변경
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
    
    def __len__(self):
        return len(self.labels)

#BERT 모델
#sentiment predict를 위해서는 num_classes를 3으로 설정해준다
#objective/subjective predict를 위해서는 num_classes를 2로 설정해준다.
#default는 3으로 되어있다
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
        
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)        


def load_model(model_path, num_classes:int = 2):
    model = BERTClassifier(bertmodel, num_classes=num_classes, dr_rate=0.5).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

class Scorer:

    def __init__(self, blogger_name, sentiment_model, subjective_model, tokenizer):
        file_path = os.path.join(root_file_path,f"{blogger_name}.csv")
        #file_path = f"./blogger detail/{blogger_name}.csv"
        self.blogger_name = blogger_name
        try:
            self.df=  pd.read_csv(file_path, index_col = 0)
        except:
            raise Exception("no blogger name")
        self.tok = tokenizer
        self.sentiment_model = sentiment_model
        self.subjective_model = subjective_model

    def row_func(self, row):
        year, month, day, _ = row.split(".")
        return datetime.date(int(year), int(month), int(day))
    
    def preprocessing(self, sentence):
        sentence = re.sub("[^ ㄱ-ㅎ가-힣a-zA-Z0-9]","",sentence)
        return sentence

    def split_post(self, post):
        sentences = []
        for sentence in kss.split_sentences(post):
            sentences.append(sentence)
        return sentences

    def bertToTorch(self, _data, device):
        token_ids, valid_length, segment_ids, label = _data
        token_ids = torch.tensor([token_ids])
        segment_ids = torch.tensor([segment_ids])
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = torch.IntTensor([valid_length.item()])
        return token_ids, valid_length, segment_ids

    def toBertDataset(self, post):
        max_len = 64
        post = [[sent,'1'] for sent in post]
        bertdataset = BERTDataset(post, 0, 1, self.tok, max_len, True, False)
        return bertdataset

    def scoring_func(self, _predicted):
        predicted = _predicted.cpu().detach().numpy()
        predicted_value = np.argmax(predicted)
        return predicted_value

    def predict_label(self, _bertdataset, _model, score_func):
        predicted_labels = []
        for bs in _bertdataset:
            token_ids, valid_length, segment_ids = self.bertToTorch(bs, device)
            predicted = _model(token_ids, valid_length, segment_ids)
            predicted_value = score_func(predicted)
            predicted_labels.append(predicted_value)
        
        predicted_labels = predicted_labels
        return predicted_labels

    def scoring(self):
        df = self.df
        dates = df['Posting Date']
        dates = dates.apply(self.row_func)

        df['Posting Date'] = dates
        df = df.sort_values(by=['Posting Date'], ascending=False)

        if len(df) > 90:
            df = df[:90]

        posts = df['Post']
        posts = posts.apply(self.preprocessing)
        df['Post'] = posts
        
        posts = df['Post'].apply(self.split_post)
        posts_sentiments = []
        posts_subjectives = []

        for post in posts:

            post_bert = self.toBertDataset(post)
            predicted_sentiments = self.predict_label(post_bert, self.sentiment_model, self.scoring_func)
            predicted_subjectives = self.predict_label(post_bert, self.subjective_model, self.scoring_func)

            posts_sentiments.append(predicted_sentiments)
            posts_subjectives.append(predicted_subjectives)

        posts = posts.values
        after_predicted = []
        for post, sentiments, subjectives in zip(posts, posts_sentiments, posts_subjectives):
            after_predicted.append([(sentence, sentiment, subjective) for sentence, sentiment, subjective in zip(post, sentiments, subjectives)])

        df['AP'] = after_predicted

        file_path = os.path.join(result_file_path,f'{self.blogger_name}.csv')
        df.to_csv(file_path)
        print(f"{self.blogger_name} saving finish!")


In [ ]:
#KoBERT 모델과 KoBERT vocabulary를 다운받는다.
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

#전체 블로거 닉네임 리스트
df = pd.read_csv('/content/drive/MyDrive/dataset/recommender_final.csv',index_col=0)
blogger_list = list(df.index)
sentiment_model = load_model(sent_path, 3)
subjective_model = load_model(subj_path, 2)

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False) 
for blogger_name in blogger_list:
    try:
        scorer = Scorer(blogger_name, sentiment_model, subjective_model, tok)
        scorer.scoring()
    except Exception as e :
        print(e)